In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
# -------------------------------------------------- GROUPING ------------------------------------------------------------
# Grouping according to jet number

def grouping(tX):
    jet_num_idx = []
    jet_num_idx.append(np.where(tX[:,22] == 0))
    jet_num_idx.append(np.where(tX[:,22] == 1))
    jet_num_idx.append(np.where(tX[:,22] == 2))
    jet_num_idx.append(np.where(tX[:,22] == 3))

    return jet_num_idx
jet_num_idx = grouping(tX)

In [4]:
np.unique(np.where(tX[jet_num_idx[0], :] == -999)[2]), np.unique(np.where(tX[jet_num_idx[1], :] == -999)[2]), np.unique(np.where(tX[jet_num_idx[2], :] == -999)[2]), np.unique(np.where(tX[jet_num_idx[3], :] == -999)[2])

(array([ 0,  4,  5,  6, 12, 23, 24, 25, 26, 27, 28]),
 array([ 0,  4,  5,  6, 12, 26, 27, 28]),
 array([0]),
 array([0]))

In [5]:
# -------------------------------------- IMPUTATION ACCORDING TO JET NUMBER ----------------------------------------------
def imputation (data, jet_num_idx):
    # Imputation the mass column with the most frequent value
    tx_imp = data.copy()
    good_idx = np.where(data[:, 0] != -999)
    round_values = np.round(data[good_idx, 0]).astype(int)
    counts = np.bincount(round_values[0,:])
    tx_imp[:, 0] = np.where(tx_imp[:, 0] == -999, np.argmax(counts), tx_imp[:, 0]) 

    # Imputation of data for jet_num = 0 and jet_num = 1
    tx_imp[jet_num_idx[0], :] = np.where(tx_imp[jet_num_idx[0], :] == -999, 0, tx_imp[jet_num_idx[0], :])
    tx_imp[jet_num_idx[1], :] = np.where(tx_imp[jet_num_idx[1], :] == -999, 0, tx_imp[jet_num_idx[1], :])

    return tx_imp
tx = imputation (tX, jet_num_idx)

In [6]:
# ----------------------------------------------- VARIANCE THRESHOLD -----------------------------------------------------------
thresh = 0
data = tx[jet_num_idx[0][0], :]
def variance(data, thresh):
    v_vector = np.var(data, axis=0)
    index_keep = np.where(v_vector > thresh)
    new_data = data[:, index_keep[0]]
    
    return new_data
new_data = variance(data, thresh)

In [7]:
# ---------------------------------------------- CORRELATION COEFFICIENT -------------------------------------------------------
thresh_corr = 0.8
data = tx[jet_num_idx[0][0], :]

def correlation(data, thresh_corr):
    corr_mat = np.empty([data.shape[1], data.shape[1]])
    for i in range(data.shape[1]):
        for j in range(i):
            if i != j:
                corr_mat[i, j] = np.corrcoef(data[:, i], data[:, j])[0, 1]

    index_out1 = np.unique(np.where(corr_mat > 0.8)[0])
    index_out2 = np.unique(np.where(corr_mat > 0.8)[1])
    all_idx = range(data.shape[1])

    if len(index_out1) > len(index_out2):
        new_data = data[:, np.setdiff1d(all_idx, index_out1)]
    else:
        new_data = data[:, np.setdiff1d(all_idx, index_out2)]
    
    return new_data
new_data2 = correlation(new_data, thresh_corr=0.8)

In [8]:
# ---------------------------------------------------- STANDARDIZE -------------------------------------------------------------
from data_processing import *
y_data = y[jet_num_idx[0][0]]
tx_std = standardize(new_data2)

#-------------------------------------------------- HANDLING OUTLIERS ----------------------------------------------------------
from data_processing import *
def remove_outliers(data, y, std_limit = 4):

    num_datapoints = np.shape(data)[0]
    num_feat = np.shape(data)[1]
    indices = np.indices((1,num_datapoints))

    standardized = standardize(tx)
    number_outliers = np.zeros((1,num_feat))
    index_outliers = []

    for ii in range(num_feat):    
        pos_outlier = standardized[:,ii]>std_limit
        neg_outlier = standardized[:,ii]<-std_limit
        number_outliers[0,ii] = np.sum(pos_outlier) + np.sum(neg_outlier)
    
        for jj in range(num_datapoints):
            if (pos_outlier[jj] == True or neg_outlier[jj] == True) and jj not in index_outliers:
                index_outliers.append(jj)

    print("Percentage of points containing at least one outlier is", f'{(100*len(index_outliers)/num_datapoints):.3f}%')
    standardized_outliers_removed = standardized[np.setdiff1d(indices,index_outliers)]
    y_std = y[np.setdiff1d(indices,index_outliers)]
    
    return standardized_outliers_removed, y_std
tx_std, y_std = remove_outliers(tx, y, std_limit = 4)

In [9]:
# Linear regression using gradient descent
from implementations import *
# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.7 # use linspace to test various gamma values and look for the best ?

# Initialization
initial_w = np.zeros(tx_std.shape[1]+1)
tx_offset = np.empty([tx_std.shape[0], tx_std.shape[1]+1])
tx_offset[:, 0] = np.ones([tx_std.shape[0]]) 
tx_offset[:, 1:] = tx_std
loss_GD, w_GD = least_squares_GD(y_data, tx_offset, initial_w, max_iters, gamma)

print("Gradient Descent: loss={l}, w = {w}".format(
    l=loss_GD, w=w_GD))


Gradient Descent: loss=42.59247529801873, w = [-0.48971605 -2.60767668 -0.71779636 -3.21291976 -1.52302557 -0.43929834
 -0.49430603 -0.34019738 -1.47949577 -0.04899586  0.02464828 -1.9731975
 -0.03676249  0.00991877  0.02442958 -0.01480398 -1.27873522]


In [10]:
# Time Visualization
# from ipywidgets import IntSlider, interact

# def plot_figure(n_iter):
#     fig = gradient_descent_visualization(
#         gradient_losses, gradient_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
#     fig.set_size_inches(10.0, 6.0)

# interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

In [11]:
# Linear regression using stochastic gradient descent

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.7 # use linspace to test various gamma values and look for the best ?

# Initialization
# initial_w = np.zeros(tx.shape[1]+1)
# tx_offset = np.empty([tx.shape[0], tx.shape[1]+1])
# tx_offset[:, 0] = np.ones([tx.shape[0]]) 
# tx_offset[:, 1:] = tx
loss_GD, w_GD = least_squares_SGD(y_data, tx_offset, initial_w, max_iters, gamma)

print("Stochastic Gradient Descent: loss={l}, w = {w}".format(
    l=loss_GD, w=w_GD))

Stochastic Gradient Descent: loss=1.7966466423537952e+39, w = [ 5.55257852e+18  1.40322112e+19  4.10112091e+18  1.33253907e+19
  3.89699408e+18 -3.49941072e+18 -4.31707316e+18 -2.95560269e+18
  1.60751829e+19 -1.26417007e+19 -5.22408568e+18  1.93740288e+19
 -1.08678164e+19  3.37890042e+18 -4.02084985e+18 -6.11775888e+18
  1.02589129e+19]


# Logistic Regression

## Functions

In [12]:
def sigmoid(t):
    z = np.squeeze(t)
    #sigma = np.where(z>=0,1/(1+np.exp(-z)),np.exp(z)/(1+np.exp(z)))
    z_bound = np.where(z<-100,-100,1)
    sigma = 1.0 / (1+np.exp(-z_bound))
    sigma = np.expand_dims(sigma, axis=1)
    return sigma

In [13]:
def calculate_logistic_loss(y, tx, w):
    """compute the loss: negative log likelihood."""
    sigma = sigmoid(tx.dot(w))
    loss = (y.T.dot(np.log(sigmoid(tx.dot(w)))) + (1-y).T.dot(np.log(1-sigmoid(tx.dot(w)))))
    return np.squeeze(-loss)

In [14]:
def calculate_logistic_gradient(y, tx, w):
    """compute the gradient of loss."""
    sigma = sigmoid(tx.dot(w))
    ty = np.expand_dims(y,axis=1)
    grad = tx.T.dot((np.subtract(sigma, ty)))
    return grad

In [15]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient"""
    penalty = np.squeeze(lambda_*w.T.dot(w))
    loss = calculate_logistic_loss(y,tx,w) + penalty 
    grad = calculate_logistic_gradient(y,tx,w) #+ 2*lambda_*w
    return loss, grad

In [16]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [17]:
def logistic_regression_GD(y, tx, gamma, lambda_, max_iter=10000):
    
    #initialise w
    w = np.zeros((tx.shape[1], 1))
    losses = []

    #logistic regression
    for iter in range(max_iter):
        
        # get loss and update w.
        loss, grad = penalized_logistic_regression(y, tx, w, lambda_)
        w = w - (gamma * grad)
        
        # decrease step size
        if iter % 10 == 0:
            gamma = gamma*0.5
            
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    l = calculate_logistic_loss(y, tx, w)
    print("Final loss={l}".format(l=calculate_logistic_loss(y, tx, w)))
    return l, w

In [18]:
def logistic_regression_SGD(y, tx, gamma, lambda_, max_iter=10000, batch_size=1):
    
    #initialise w
    w = np.zeros((tx.shape[1], 1))
    losses = []

    #logistic regression
    for iter in range(max_iter):
        
        # get loss and update w.
        for y_b, tx_b in batch_iter(y, tx, batch_size):
            loss, grad = penalized_logistic_regression(y, tx, w, lambda_)
            w = w - (gamma * grad)
        
        # decrease step size
        if iter % 10 == 0:
            gamma = gamma*0.5
            
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    l = calculate_logistic_loss(y, tx, w)
    print("Final loss={l}".format(l=calculate_logistic_loss(y, tx, w)))
    return l, w

## Logistic Cross validation

In [73]:
def build_k_indices(y,k_fold,seed):
    
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    
    return np.array(k_indices)

def logistic_cross_validation(y, x, k_indices, k, gamma, lambda_):
   
    train_indices = np.setdiff1d(k_indices,k_indices[k])
    
    x_test = x[k_indices[k],:]
    x_train = x[train_indices,:]
    y_test = y[k_indices[k]]
    y_train = y[train_indices]

    loss_train,w_optimal = logistic_regression_GD(y_train,x_train, gamma, lambda_, max_iter=10000)
    pred = predict_labels(w_optimal,x_test)
    pred_bin = np.squeeze(1*np.equal(pred,1))
    correct = np.sum(1*np.equal(y_test,pred_bin))
    all_ = np.shape(y_test)[0]
    acc =  correct/all_
    print(f"The accuracy is",acc) 

    return loss_train, acc

def logistic_lambda_optimisation(y,x,k,gamma,seed=1):
    
    lambdas = np.logspace(-4, 0, 30)
    k_indices = build_k_indices(y,k,seed)
    accuracy = []
    
    for lambda_ in lambdas:
        accuracy_intermediate = []
    
        for ii in range(k):
            print(f"Doing fold",ii,"for lambda value",lambda_) 
            loss_tr_ii, acc_test = logistic_cross_validation(y, x, k_indices, ii, gamma, lambda_)
            accuracy_intermediate.append(acc_test)
        
        accuracy.append(np.mean(accuracy_intermediate))
    
    idx_min_acc = np.argmin(accuracy)
    lambda_optimal = lambdas[idx_min_acc]
    
    return lambda_optimal
    

In [74]:
y = np.expand_dims(y, axis=1)
y_bin = 1*np.equal(y_data,1)
np.shape(tx_std)
threshold = 1e-3

In [75]:
#y_sqeezed = np.squeeze(y_data)
#y_bin_squeezd = np.squeeze(y_bin)
np.sum(1*np.equal(y_data,y_bin))

25492

In [76]:
lbd = logistic_lambda_optimisation(y_bin, tx_offset, 5,0.001)
print(lbd)

Doing fold 0 for lambda value 0.0001
Current iteration=0, loss=84602.38015995664
Final loss=543831.4691741293
The accuracy is 0.7435191672505255
Doing fold 1 for lambda value 0.0001
Current iteration=0, loss=84670.38015995662
Final loss=546908.3358371903
The accuracy is 0.7401161044940446
Doing fold 2 for lambda value 0.0001
Current iteration=0, loss=84489.38015995664
Final loss=542397.8319038821
The accuracy is 0.749174256831148
Doing fold 3 for lambda value 0.0001
Current iteration=0, loss=84550.38015995664
Final loss=547697.578137068
The accuracy is 0.7461215093584226
Doing fold 4 for lambda value 0.0001
Current iteration=0, loss=84563.38015995664
Final loss=546076.1924734428
The accuracy is 0.7455209688719848
Doing fold 0 for lambda value 0.00013738237958832623
Current iteration=0, loss=84602.38015995664
Final loss=543831.4691741293
The accuracy is 0.7435191672505255
Doing fold 1 for lambda value 0.00013738237958832623
Current iteration=0, loss=84670.38015995662
Final loss=546908.3

## Model optimisation

In [ ]:
    # init parameters
    max_iter = 10000
    batch_size = 1000
    gamma = 0.001
    lambda_ = 0.5
    threshold = 1e-3
    losses = []

In [ ]:
l_log_SGD, w_log_GD = logistic_regression_SGD(y_bin, tx_offset, gamma, lambda_, max_iter, batch_size)
print("Stochastic Gradient Descent: loss={l}, w = {w}".format(
    l=l_log_SGD, w=w_log_GD))

In [ ]:
fin_losses = []
test_val = [1e-5,1e-4,1e-3,1e-2,1e-1,1,1e1,1e2]
for i in test_val:
    lambda_ = i
    model, final_loss = logistic_regression_GD(y_bin, tx_offset, gamma, lambda_, batch_size, max_iter,stoch=False)
    fin_losses.append(final_loss)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
# def imputation(tX):
#     # Imputation Using Zero
#     tx_zeros = np.where(tX == -999, 0, tX) 

#     # Imputation using the most frequent value (constant)
#     col_index = np.unique(np.where(tX == -999)[1])
#     tx_mostFrq = tX.copy()
#     for index in col_index:
#         good_idx = np.where(tX[:, index] != -999)
#         round_values = np.round(tX[good_idx, index]).astype(int)
    
#         # Taking care of the negative values
#         neg = round_values[np.where(round_values < 0)]  
#         if neg.size > 0:    
#             # Positive values
#             pos = round_values[np.where(round_values >= 0)]        
#             counts_neg = np.bincount(np.abs(neg))
#             counts_pos = np.bincount(pos)
        
#             if max(counts_neg) > max(counts_pos):
#                 tx_mostFrq[:, index] = np.where(tx_mostFrq[:, index] == -999, -np.argmax(counts_neg), tx_mostFrq[:, index])
        
#             else:
#                 tx_mostFrq[:, index] = np.where(tx_mostFrq[:, index] == -999, np.argmax(counts_pos), tx_mostFrq[:, index])
#         else:
#             counts = np.bincount(round_values[0,:])
#             tx_mostFrq[:, index] = np.where(tx_mostFrq[:, index] == -999, np.argmax(counts), tx_mostFrq[:, index]) 
            
#     tx_mixed = tX.copy()
#     tx_mixed[:, col_index[0]] = tx_mostFrq[:, col_index[0]]
#     tx_mixed[:, col_index[1:]] = tx_zeros[:, col_index[1:]]
    
#     return tx_zeros, tx_mostFrq, tx_mixed